# Basics of blockchain

In this notebook I set out to create a simple blockchain for a basic payment system. I've followed these notes [here](https://github.com/emunsing/tutorials/blob/master/BuildYourOwnBlockchain.ipynb)

Alex and Sam are the two parties involved in transactions.

First I make a hashing function that will take a message string and pass it through the sha256 algorithm. This will return the utf encoded version of the hash. hexdigest returns the digest of the string.

Hashing is great because it takes plain text and outputs a fixed length string which is encrypted. This is one way and changing even just 1 character of the message string will alter the whole hash output. This is the Crypto aspect of the crypto currency.

In [3]:
import hashlib, json, sys

def hashmsg(msg=""):
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)
        print(msg)
    if sys.version_info.major==2:
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()


In [4]:
hashmsg("THIS IS A SECRET MESSAGE NO ONE WILL EVER EVER CRACK")

hashmsg("Sam pays alex 2 tokens")

'13b461d73e663cc26b0f76cd85cd8b6e070d62f7bb5f8c7d565c7d6f2d935bcb'

Now we need a function to make some random transactions. We use random.getrandbits to get a random long int with k bits.  A bit is simply a 0 or a 1. We multiply by 2 and take away -1 so the number is either 1 or -1. Negative represents a withdrawal and positive reprsents a deposit.

In [23]:
import random
random.seed(0)

def makeTransaction(maxValue=3):
    sign=int(random.getrandbits(1))*2-1
    #print(sign)
    amount=random.randint(1,maxValue)
    samPays= sign*amount
    alexPays=-sign*amount
    
    return {u'Alex':alexPays,u'Sam': samPays}
    


makeTransaction()

{'Alex': -2, 'Sam': 2}

Let's make 30 transactions using the function above.

In [29]:
tnxBuffer=[makeTransaction() for i in range(30)]
print(tnxBuffer)

[{'Alex': -1, 'Sam': 1}, {'Alex': 3, 'Sam': -3}, {'Alex': -3, 'Sam': 3}, {'Alex': 2, 'Sam': -2}, {'Alex': 1, 'Sam': -1}, {'Alex': -1, 'Sam': 1}, {'Alex': -1, 'Sam': 1}, {'Alex': -1, 'Sam': 1}, {'Alex': -1, 'Sam': 1}, {'Alex': -3, 'Sam': 3}, {'Alex': -3, 'Sam': 3}, {'Alex': -2, 'Sam': 2}, {'Alex': -3, 'Sam': 3}, {'Alex': -2, 'Sam': 2}, {'Alex': -1, 'Sam': 1}, {'Alex': -1, 'Sam': 1}, {'Alex': -3, 'Sam': 3}, {'Alex': -2, 'Sam': 2}, {'Alex': -2, 'Sam': 2}, {'Alex': 2, 'Sam': -2}, {'Alex': -3, 'Sam': 3}, {'Alex': -3, 'Sam': 3}, {'Alex': -2, 'Sam': 2}, {'Alex': 2, 'Sam': -2}, {'Alex': -1, 'Sam': 1}, {'Alex': 3, 'Sam': -3}, {'Alex': -2, 'Sam': 2}, {'Alex': -1, 'Sam': 1}, {'Alex': 1, 'Sam': -1}, {'Alex': -2, 'Sam': 2}]


Above are all our transactions. We will take the first k transactions from the buffer above and chunk them into blocks.

However before allowing transactions into the block we must validate them in some way and make sure they haven't been forged. We'll allow for some simpe rules. In Ethereum, the validation function checks that the smart contracts were faithfully executed and respect gas limits.

1. The sum of deposits and withdrawals must be equal. We can't have tokens created or destroyed.
2. Alex or Sam must have sufficient funds to cover withdrawals. Money doesn't grow on trees!

The next function we define will compute the current state based on a transaction. I.e. we pass in a transactions and update the parties balances. Sam's current state is 5 tokens, he pays Alex 3 therefore his state in the next period becomes 2.

In [30]:
#pass a transaction and the current state then for each key in the transaction and then each key in the state update.
def updateState(txn,state):
    
    state=state.copy()
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else: 
            state[key] = txn[key]
        
    return state

In [31]:
def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction does not cause an overdraft
    #txn.keys() returns the parties involved in transaction
    for key in txn.keys():
        if key in state.keys(): 
            #pass current state balance into accountbalance variable
            acctBalance = state[key]
        #else set their account balance is 0 they don't have any tokens
        else:
            acctBalance = 0
        #if their current balance + whatever they are paying takes them into the negative then return false as invalid
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

After defining a isvalid function we can pass the current state and check the transaction

In [32]:
#THERE MUST BE SOME CURRENT STATE SET. IN THIS EXAMPLE SAM AND ALEX HAVE 5 TOKENS.

state={u'Sam':5,u'Alex':5}

print(isValidTxn({u'Sam': -3,  u'Alex': 3},state))  # Basic transaction- this works and prints TRUE. Sam pays alex 3.
print(isValidTxn({u'Sam': -6, u'Alex': 6},state))  # Basic transaction- this doesn't work Sam only has 5 tokens so can't pay 6
print(isValidTxn({u'Sam': -4, u'Alex': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Sam': -4, u'Alex': 2,'Ben':2},state)) # Creating new users is valid




True
False
False
True


## Make the blockchain

Now we have a way over updating current state and checking transactions we can start building blocks of transactions.
We must start with some state of our users therefore the first block in the chain is called the 'genesis' block.
Below initiates the first block with 1 transaction.

We store this in an array which is essentially our chain.

In [33]:
state = {u'Alex':50, u'Sam':50}  # Define the initial state Sam and Alex now have 50 tokens each.
genesisBlockTxns = [state]
print(genesisBlockTxns)
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
#Return hash of current block
genesisHash = hashmsg( genesisBlockContents )
#create block with contents and a hash. In reality this is the merkle root of all transactions in the block.
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

[{'Alex': 50, 'Sam': 50}]
{"blockNumber": 0, "parentHash": null, "txnCount": 1, "txns": [{"Alex": 50, "Sam": 50}]}


In [34]:
#This is now the original block and the chain becomes a python list.
chain = [genesisBlock]

In [35]:
def makeBlock(txns,chain):
    #get the parent
    parentBlock = chain[-1]
    #declare the parent block hash
    parentHash  = parentBlock[u'hash']
    #get new block number i.e. previous block was block 5 therefore we are making 6.
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    #how many transactions
    txnCount    = len(txns)
    #Create the block contents.
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                     u'txnCount':len(txns),'txns':txns}
    #Create the hash. In reality this is the merkle root of the hashes of transactions.
    blockHash = hashmsg( blockContents )
    #Finally construct the block object.
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block

we generated 30 transactions and we'll fix the number of transactions per block.

In [36]:
blockSizeLimit = 5  # Arbitrary number of transactions per block- 
               #  this is chosen by the block miner, and can vary between blocks!

while len(tnxBuffer) > 0:
    bufferStartSize = len(tnxBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    #while the length of transactions is >0 and less than the block size
    while (len(tnxBuffer) > 0) & (len(txnList) < blockSizeLimit):
        #.pop() returns last element
        newTxn = tnxBuffer.pop()
        validTxn = isValidTxn(newTxn,state) # This will return False if txn is invalid
        
        if validTxn:           # If we got a valid state, not 'False' then add the transaction to the list
            txnList.append(newTxn)
            state = updateState(newTxn,state) #get the current state.
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue  # This was an invalid transaction; ignore it and move on
        
    ## Make a block
    myBlock = makeBlock(txnList,chain)
    #append it to list
    chain.append(myBlock)

{"blockNumber": 1, "parentHash": "1b872ea2bf797407d01252a36f4056105e9e2a7e8c6a3a8e07e76bb09d498f24", "txnCount": 5, "txns": [{"Alex": -2, "Sam": 2}, {"Alex": 1, "Sam": -1}, {"Alex": -1, "Sam": 1}, {"Alex": -2, "Sam": 2}, {"Alex": 3, "Sam": -3}]}
{"blockNumber": 2, "parentHash": "fdcc4a9915dcc23486a49c58c4a91ce9d12f84bbe6999d22b86fba9445e2a96c", "txnCount": 5, "txns": [{"Alex": -1, "Sam": 1}, {"Alex": 2, "Sam": -2}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}, {"Alex": -3, "Sam": 3}]}
{"blockNumber": 3, "parentHash": "2c07e3fabbefb328e80a5575ade2208796713f198b5062dad3fe10f846e636e6", "txnCount": 5, "txns": [{"Alex": 2, "Sam": -2}, {"Alex": -2, "Sam": 2}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}, {"Alex": -1, "Sam": 1}]}
{"blockNumber": 4, "parentHash": "d725f99103c7edce2406eb81267ecd47b47f13d94c03ff82a17a4b04554fb982", "txnCount": 5, "txns": [{"Alex": -1, "Sam": 1}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}]}
{"blockNumber": 

WOO now we have a chain of blocks! Usually the names of transaction parties will be hash there may be other meta information in here that the miners have added.

In [63]:
chain[2]

{'contents': {'blockNumber': 2,
  'parentHash': '00be878dae4c9f475462a72a13b791a4fec373acc11dedb54c5c4fe9b186ad44',
  'txnCount': 5,
  'txns': [{'Alex': -2, 'Sam': 2},
   {'Alex': 2, 'Sam': -2},
   {'Alex': -1, 'Sam': 1},
   {'Alex': -2, 'Sam': 2},
   {'Alex': 1, 'Sam': -1}]},
 'hash': '920016495ccbd5a6bfd73cff88c2ed8b2a809189e699f3cd44f15f86386ab0ad'}

We can also check the state after blocking those 30 transactions. Alex has 20 tokens and Sam has 80. They both started with 50 each in the genesis block.

In [37]:
state

{'Alex': 20, 'Sam': 80}

## Checking Chain Validity

Now we've defined and chained blocks together we need to check the chain is valid. Obviously the block chain is hosted on a number of nodes and if they don't all match then some version must be invalid. Someone could of tampered to try and fake their current state. Once our node is synced with the network (has an up-to-date copy of the blockchain and a representation of system state) it will need to check the validity of new blocks that are broadcast to the network.

We'll need three functons

1. Check the hash of each block is right, so take the transactions and see if the hash matches what's in there.
2. Check validity of a block given it's parent and current.
3.  Check the validity of the entire chain, and compute the system state beginning at the genesis block. This will return the system state if the chain is valid, and raise an error otherwise. i.e. the genesis state was 50, 50 and after transactions should be 20,80. If it's not something is wrong.


In [38]:
#Pretty self explanatory this one
def checkBlockHash(block):
    # Raise an exception if the hash does not match the block contents
    expectedHash = hashmsg( block['contents'] )
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

In [39]:
def checkBlockValidity(block,parent,state):    
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    # Check transaction validity; throw an error if an invalid transaction was found.
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

In [40]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment), 
    #  checking that all transactions are internally valid,
    #    that the transactions do not cause an overdraft,
    #    and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    #   or returns False if an error was detected

    
    ## Data input processing: Make sure that our chain is a list of dicts
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {}
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    ## Checking subsequent blocks: These additionally need to check
    #    - the reference to the parent block's hash
    #    - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state

In [41]:
checkChain(chain)

{"blockNumber": 0, "parentHash": null, "txnCount": 1, "txns": [{"Alex": 50, "Sam": 50}]}
{"blockNumber": 1, "parentHash": "1b872ea2bf797407d01252a36f4056105e9e2a7e8c6a3a8e07e76bb09d498f24", "txnCount": 5, "txns": [{"Alex": -2, "Sam": 2}, {"Alex": 1, "Sam": -1}, {"Alex": -1, "Sam": 1}, {"Alex": -2, "Sam": 2}, {"Alex": 3, "Sam": -3}]}
{"blockNumber": 2, "parentHash": "fdcc4a9915dcc23486a49c58c4a91ce9d12f84bbe6999d22b86fba9445e2a96c", "txnCount": 5, "txns": [{"Alex": -1, "Sam": 1}, {"Alex": 2, "Sam": -2}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}, {"Alex": -3, "Sam": 3}]}
{"blockNumber": 3, "parentHash": "2c07e3fabbefb328e80a5575ade2208796713f198b5062dad3fe10f846e636e6", "txnCount": 5, "txns": [{"Alex": 2, "Sam": -2}, {"Alex": -2, "Sam": 2}, {"Alex": -2, "Sam": 2}, {"Alex": -3, "Sam": 3}, {"Alex": -1, "Sam": 1}]}
{"blockNumber": 4, "parentHash": "d725f99103c7edce2406eb81267ecd47b47f13d94c03ff82a17a4b04554fb982", "txnCount": 5, "txns": [{"Alex": -1, "Sam": 1}, {"Alex": -2, "Sam": 2}, 

{'Alex': 20, 'Sam': 80}

We've checked the chain and the correct state after all 30 transactions is correct.

# Putting it together, final architecture

In an actual block chain network new nodes download a copy of the latest full block chain and verify it as we did above. They announce their present on the network and start listening for transactions. Once they've bundled transactions together they pass the block onto other nodes.

Below we act as if we are a new node. We take a copy of the latest chain we got and then generate 5 more transactions. We then run our functions to make the next block.

In [42]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

{"blockNumber": 7, "parentHash": "68b0e32d1b6b991c4f91f64440cb6b165f70c2063847ab72adcb415ade126d3b", "txnCount": 5, "txns": [{"Alex": 3, "Sam": -3}, {"Alex": 2, "Sam": -2}, {"Alex": -1, "Sam": 1}, {"Alex": 2, "Sam": -2}, {"Alex": -1, "Sam": 1}]}


In [43]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 7 blocks long
New Block Received; checking validity...
{"blockNumber": 7, "parentHash": "68b0e32d1b6b991c4f91f64440cb6b165f70c2063847ab72adcb415ade126d3b", "txnCount": 5, "txns": [{"Alex": 3, "Sam": -3}, {"Alex": 2, "Sam": -2}, {"Alex": -1, "Sam": 1}, {"Alex": 2, "Sam": -2}, {"Alex": -1, "Sam": 1}]}
Blockchain on Node A is now 8 blocks long


We have covered some basic architecture for how a blockchain might work. We now have functions to make transactions, blocks and check validity of the block and the chain.

We can also derive the current state from the chain. This is important to know how everyone stands.

Here's stuff we haven't covered...

* why miners exist
* the nonce
* merkle roots
* hashcash
* public key cryptography
* block consensus

# References

https://github.com/emunsing/tutorials/blob/master/BuildYourOwnBlockchain.ipynb
https://blockexplorer.com/
https://github.com/bitcoin/bitcoin

https://www.youtube.com/watch?v=gUwXCt1qkBU - GREAT VIDEO ON MERKLE ROOTS